# Mean traces

In this notebook we will reproduce part of Figure 2 in the paper which compted the average displacement and velocity and performs a background correction of the traces.
First we import the necessary packages

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
import ap_features as apf
import numpy as np
import mps
import mps_motion
from mps_motion import (
    Mechanics,
    OpticalFlow,
)


import dataset

And load the sample dataset

In [ ]:
full_resolution = False
data = dataset.load_sample_data(full_resolution=full_resolution)

Next we will create an optical flow object and specify that we want to use the Farnebäck method

In [ ]:
opt_flow = mps_motion.OpticalFlow(data, flow_algorithm="farneback")

Now we will compute the displacements

In [ ]:
u = opt_flow.get_displacements()

Lets us inspect the object we got back first

In [ ]:
u

We see that we got a `VectorFrameSequence`, which is essentially a sequence of frames with a vector, representing the x and y component of the displacement at each point. To convert this into a trace we need to reduce it along some dimension. For example, we could take the norm at each pixel

In [ ]:
u_norm = u.norm()
u_norm

Which gives us a `FrameSequence`, i.e a sequence of frames with a scalar value at each pixel. Next thing we will do is to reduce each frame to a number, and we can do this by takeing the average value

In [ ]:
u_norm_mean = u_norm.mean()
u_norm_mean

As we can see we get a [`dask` array](https://docs.dask.org/en/stable/array.html) back. These are similar to numpy array, but much more memory efficient and we can also perform operations on them in parallel. To get the actual value, we need to call compute.

In [ ]:
u_norm_mean_array = u_norm_mean.compute()
u_norm_mean_array

We can now plot the array

In [ ]:
fig, ax = plt.subplots()
ax.plot(data.time_stamps, u_norm_mean_array)
ax.set_xlabel("Time (s)")
ax.set_ylabel("Displacement (\u00B5m)")
plt.show()

Note that in this case we didn't specify which frame to use as the reference frame, which means that it will use the first frame. This turns out to work well in this case, but image we for example had chosen frame at time 500 ms (which is approximately at the peak contraction), then we would get

In [ ]:
u_20 = opt_flow.get_displacements(reference_frame=500, recompute=True)
u_20_array = u_20.norm().mean().compute()
fig, ax = plt.subplots()
ax.plot(data.time_stamps, u_20_array)
ax.set_ylabel("Displacement (\u00B5m)")
ax.set_xlabel("Time (s)")
plt.show()

We will see how we can automate the detection of a suitable reference frame later.

First we can also find the velocity trace

In [ ]:
v = opt_flow.get_velocities()
v_norm_mean_array = v.norm().mean().compute()

time = data.time_stamps[:-1]

fig, ax = plt.subplots()
ax.plot(time, v_norm_mean_array)
ax.set_xlabel("Time (s)")
ax.set_ylabel("Velocity (\u00B5m/s)")
plt.show()

Here it is also worth noting that we have an array that has one less element than the displacement, because the velocity is essentially computing be taking the difference between successive displacements and dividing by the time difference. We could also increase the spacing between the displacement by passing in the spacing argument

In [ ]:
v5 = opt_flow.get_velocities(spacing=5)
v5_norm_mean_array = v5.norm().mean().compute()

time = data.time_stamps[:-5]

fig, ax = plt.subplots()
ax.plot(time, v5_norm_mean_array)
ax.set_xlabel("Time (s)")
ax.set_ylabel("Velocity (\u00B5m/s)")
plt.show()

As a consequence we get a more smooth signal. Depending on the framerate of the video you are analyzing you might choose different spacing to avoid too much noise in the velocity traces. For this dataset a spacing of 1.0 seems to be appropriate.

## Estimating the reference frame
We would like to choose a reference frame where the cells are at rest, which would be equivalent to having (close to) zero velocity. For this we have implemented a helper function to achieve just that.

In [ ]:
ref_index = mps_motion.motion_tracking.estimate_referece_image_from_velocity(t=data.time_stamps[:-1], v=v_norm_mean_array)
print(f"Reference index {ref_index} at time {data.time_stamps[ref_index]}")
type(ref_index)

And we see that it found a reference index of 161 and time 4025

In [ ]:
fig, ax = plt.subplots()
ax.plot(data.time_stamps[:-1], v_norm_mean_array)
ax.plot([data.time_stamps[ref_index]], [v_norm_mean_array[ref_index]], "ro")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Velocity (\u00B5m/s)")
plt.show()

And we can now use this reference index instead for the displacement

In [ ]:
u = opt_flow.get_displacements(reference_frame=data.time_stamps[ref_index], recompute=True)
u_array = u.norm().mean().compute()
fig, ax = plt.subplots()
ax.plot(data.time_stamps, u_array)
ax.set_ylabel("Displacement (\u00B5m)")
ax.set_xlabel("Time (s)")
plt.show()

## Background correction

Run running the motion tracking software, some background drift might be cumulated in the resulting displacements. However the displacements at the reference image should be zero. This might lead to a sudden jump in the displacement at the point of reference. To circumvent this, we have an argument called `smooth_ref_transition`, which by default is set to True. If we set this to false, we see this phenomena more clearly

In [ ]:
u_no_smooth = opt_flow.get_displacements(reference_frame=data.time_stamps[ref_index], recompute=True, smooth_ref_transition=False)
u_no_smooth_array = u_no_smooth.norm().mean().compute()
fig, ax = plt.subplots()
ax.plot(data.time_stamps, u_no_smooth_array)
ax.set_ylabel("Displacement (\u00B5m)")
ax.set_xlabel("Time (s)")
plt.show()

To remove the background from the trace, we can use a [background correction algorithm](https://doi.org/10.1016/j.chemolab.2004.10.003) to first estimate the baseline and then subtract it. This algorithm is implemented in the `ap_features` library

In [ ]:
background = apf.background.correct_background(data.time_stamps, u_array, method="subtract")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 3))
ax[0].plot(data.time_stamps, u_array)
ax[0].plot(data.time_stamps, background.background)
ax[0].set_ylabel("Displacement (\u00B5m)")
ax[0].set_title("Original with backgorund")
ax[1].plot(data.time_stamps, background.corrected)
ax[1].set_title("Corrected")

In this base there is not much drift in the signal, but this might occur in other datasets. 